# A* :

The A* (pronounced "A-star") search algorithm is a widely used and effective pathfinding algorithm. It's commonly used in various fields, including artificial intelligence, robotics, and video games. A* is an extension of dijkstra algorithm and is used to find the shortest path from a starting point to a goal in a graph, grid, or map.

Here's a high-level explanation of the A* search algorithm:

1. **Initialization:**

   - You start with a graph or grid representing your environment. Each location in this environment is called a "node" or "cell."
   - You have a starting node and a goal node. The goal is the destination you want to reach.

2. **Open List and Closed List:**

   - A* uses two lists: the "Open List" and the "Closed List."
   - The Open List holds nodes that need to be explored or considered.
   - The Closed List holds nodes that have already been explored.

3. **G and H Scores:**

   - Each node on the Open List has two values associated with it: G and H scores.
   - G (the "cost-so-far") represents the actual cost to reach a particular node from the starting node.
   - H (the "heuristic") is an estimate of the cost from the current node to the goal. It's a heuristic because it doesn't give you the exact cost but an estimation.

4. **F Score:**

   - The F score is the sum of G and H: `F = G + H`.
   - A* selects the node with the lowest F score from the Open List, as this is the node that is estimated to be the most promising.

5. **Expanding Nodes:**

   - For the selected node, you expand it by considering its neighbors (often called "successor" nodes).
   - Calculate G and H scores for each neighbor.
   - If a neighbor is not on the Open or Closed List, add it to the Open List with the calculated F, G, and H scores.

6. **Choosing the Next Node:**

   - The node with the lowest F score on the Open List becomes the next node to explore.
   - If the goal node is selected, the path is found, and you can reconstruct the path by tracing back from the goal node to the start node.

7. **Repeat:**

   - Continue this process until you reach the goal node, or the Open List becomes empty (indicating that there is no path to the goal).

8. **Optimality:**

   - A* is known for its optimality; it guarantees that it will find the shortest path if one exists.

A* uses a combination of the G and H scores to prioritize exploring nodes. The G score ensures that the algorithm explores paths with lower actual costs, and the H score guides it toward the goal. By considering the total cost (F score), A* efficiently finds the optimal path. However, it's important to choose an appropriate heuristic for the problem to ensure A* works effectively.


Inspired by [Daniel Martinex Bielostotzky](https://www.kaggle.com/danielmartinezb) the problem to ensure A* works effectively.#

In [1]:
# import the only module needed
from simpleai.search import SearchProblem, astar

### Data structure
To represent the irregular 6x6 sudoku board, chains are used, specifically two, one that will represent the numbers on the board and another that will represent the structure of the board (the shape of the subgroups)
 As an example, in this document we work with the following boad:
<div style='text-align: center';> <img src='https://i.imgur.com/ckQ8rn7.png' alt='Alt Text'  width='50%'> </div>
>
>
:

The represention of the numbers of the above table would be:

In [2]:
initial_board_str = '''0-0-0-0-1-0
0-3-0-0-2-0
0-2-0-0-0-0
0-0-0-0-5-0
0-0-0-1-0-0
0-6-5-2-0-0'''

initial_board_str

'0-0-0-0-1-0\n0-3-0-0-2-0\n0-2-0-0-0-0\n0-0-0-0-5-0\n0-0-0-1-0-0\n0-6-5-2-0-0'

In [3]:
#Randomly defining the jigsaw labelling

initial_group_str = '''1-1-1-1-1-2
3-3-4-1-2-2
3-4-4-4-2-2
3-3-4-4-2-6
3-5-6-6-6-6
5-5-5-5-5-6'''

initial_group_str

'1-1-1-1-1-2\n3-3-4-1-2-2\n3-4-4-4-2-2\n3-3-4-4-2-6\n3-5-6-6-6-6\n5-5-5-5-5-6'

The number assigned to each group does not affect the result of the algorithm, it is not even necessary for the numbers used to be consecutive, since they are used by the algorithm only to be able to group the boxes that belong to each group

# Sudoku Utilities
Here, you will find a collection of functions and subroutines that the algorithm utilizes for managing states in the methods essential to enact the solution as a search problem. These utility functions are stored within the `sudokutils.py` file within the source code available on [GitHub](https://github.com/Bielos/irregular-sudoku-a-star).

In [4]:
# Functions 'string_to_list' and 'list_to string' allows us to convert the state chain representation to matrices to make it easier for us to handle the data structure.

def string_to_list(my_string):
    return [row.split('-') for row in my_string.split('\n')]

def list_to_string(my_list):
    return('\n'.join(['-'.join(row) for row in my_list])) #fyi, append is for list and join is for strings.

In order to control the empty spaces, the '0' in the space is converted to 'X'. We will form a function which will do the same and return the position of the cell where 'X' is located.

In [5]:
# Find the location of the actual position piece in the puzzle.
# Returns a tuple: row, column

def find_actual_position(board):
    rows = string_to_list(board)
    for ir, row in enumerate(rows):
        for ic, element in enumerate(row):
            if element == 'X':
                return ir, ic

In [6]:
# This function receives a board and a row, and returns a list of valid numbers for this row.
# Find all valid numbers inside a row.
# Returns a list of strings

def possible_nos_in_row(board, a_row):
    rows = string_to_list(board)
    elements = []
    for ir, row in enumerate(rows):
        for ic, element in enumerate(row):
            if ir == a_row:
                elements.append(element)

    results = []
    for i in range(1,7):
        if str(i) not in elements:
            results.append(str(i))

    return results

In [7]:
# Create a function which receives a board and a column, and returns a list of valid numbers for this column.

def possible_nos_in_col(board, a_col):
    rows = string_to_list(board)
    elements = []
    for ir, row in enumerate(rows):
        for ic, element in enumerate(row):
            if ic == a_col:
                elements.append(element)
    
    results = []
    for i in range(1,7):
        if str(i) not in elements:
            results.append(str(i))

    return results

In [8]:
# This function receives a board and a group, and returns a list of valid numbers for this group.
# groups: A string representing groups on the board (e.g., rows, columns, or other segments).
# group: An integer representing the group number for which you want to find valid numbers.
def possible_nos_in_group(board, groups, group):
    board_list = string_to_list(board)
    rows = string_to_list(groups)
    elements = []

    for ir, row in enumerate(rows):
        for ic, gr in enumerate(row):
            if gr == str(group):
                elements.append(board_list[ir][ic])       

    results = []
    for i in range(1,7):
        if str(i) not in elements:
            results.append(str(i))

    return results

In [9]:
# This function returns a boolean indicating whether a board received by parameter contains empty cells or not. Returns true if there is no empty cells in a board.

def completed(board, groups):
    rows = string_to_list(board)
    for ir, row in enumerate(rows):
        for ic, element in enumerate(row):
            if element == '0' or element == 'X':
                return False
    return True

In [10]:
# /This function is one of the heuristics used for the solution, it runs through a board and returns the position of the box with the least amount of valid numbers per row, column and group.
# 'Find the next cell to expand based on the number of posible numbers in the cell. Returns a tuple: row, column'

def find_next(board, groups):
    min_possiblities = 16 #to ensure that it starts as a value greater than the maximum number of possible possibilities for a cell. for a 6x6 size, max possible posiblities would be 6
    next_actual_row = None
    next_actual_col = None

    groups_list = string_to_list(groups)
    rows = string_to_list(board)
    for ir, row in enumerate(rows):
        for ic, element in enumerate(row):
            if element == '0':
                nums_in_rows = possible_nos_in_row(board, int(ir))
                nums_in_cols = possible_nos_in_col(board, int(ic))
                nums_in_grps = possible_nos_in_group(board, groups, int(groups_list[ir][ic]))
                possiblities = [value for value in nums_in_cols if value in nums_in_rows and value in nums_in_grps] #store the numbers that are possible candidates for a particular cell on the game board
                if len(possiblities) < min_possiblities:
                    next_actual_row = ir
                    next_actual_col = ic
                    min_possiblities = len(possiblities)
                    
    return next_actual_row,next_actual_col

## Approach to the puzzle as a search problem
Here, agent was provided with additional knowledge about his environment: the validity of a number in a certain box. This would ensure the reduction of repeated states.
Additionally, with this knowledge, a heuristic was created for the location of the next box that would be marked with 'X', instead of going from left to right and up to down like in classical approach, the agent now goes around the entire board and chooses the box whose number of numbers possible is the smallest, in this way, the number of nodes in the tree is reduced to a maximum, improving the complexity in time and space of the A* algorithm for this puzzle.

## Implementation of the search problem
To implement the above definition with SimpleAI, you need to create a class that is a child of SimpleAI's SearchProblem and you will need to implement the actions, result, is_goal and heuristic methods. If the search algorithm needs a cost function, the cost method must also be implemented.

For more information on these methods, see SimpleAI's documentation on defining search problems.

For this case, the class created will be named IrregularSudokuProblem and, since A* will be used, the cost function is implemented in such a way that it always returns the constant 1.

In [11]:
class sudoku_solver(SearchProblem):
    def actions(self, state):
        actual_row, actual_col = find_actual_position(state) #state is the current state of the sudoku
        nums_in_rows = possible_nos_in_row(state, actual_row)
        nums_in_cols = possible_nos_in_col(state, actual_col)
        nums_in_grps = possible_nos_in_group(state, initial_group_str, int(groups_list[actual_row][actual_col]))
        posibilites = [value for value in nums_in_cols if value in nums_in_rows and value in nums_in_grps]
        return posibilites

    def result(self, state, action):
        actual_row, actual_col = find_actual_position(state)
        state_list = string_to_list(state)
        state_list[actual_row][actual_col] = action
        if not completed(list_to_string(state_list), initial_group_str):
            next_actual_row, next_actual_col = find_next(state, initial_group_str)
            state_list[next_actual_row][next_actual_col] = 'X'

        return list_to_string(state_list)
    def is_goal(self, state):
        return completed(state,initial_group_str)

    def cost(self, state, action, state2):
        return 1
    
    def heuristic(self, state):
        # how many empty cells are we to complete?
        h = 0
        rows = string_to_list(state)
        for ir, row in enumerate(rows):
            for ic, element in enumerate(row):
                if element == '0':
                    h = h + 1
        return h 

In [12]:
#global variable definition
groups_list = string_to_list(initial_group_str)

#initial state definition
initial_state_list = string_to_list(initial_board_str)
next_actual_row, next_actual_col = find_next(initial_board_str, initial_group_str)
initial_state_list[next_actual_row][next_actual_col] = 'X'

initial_state = list_to_string(initial_state_list) 

#instance of the problem with the initial state
my_problem = sudoku_solver(initial_state=initial_state)

# A* implementation and results
result = astar(my_problem)
for action, state in result.path():
    print('Insert number', action)
print('-------------')
print('FINAL STATE')
print('-------------')
print(state)

Insert number None
Insert number 4
Insert number 5
Insert number 1
Insert number 3
Insert number 6
Insert number 6
Insert number 1
Insert number 4
Insert number 4
Insert number 1
Insert number 4
Insert number 4
Insert number 3
Insert number 5
Insert number 5
Insert number 6
Insert number 6
Insert number 1
Insert number 3
Insert number 2
Insert number 6
Insert number 4
Insert number 2
Insert number 3
Insert number 3
Insert number 2
Insert number 5
-------------
FINAL STATE
-------------
3-5-2-4-1-6
5-3-4-6-2-1
6-2-1-5-4-3
4-1-6-3-5-2
2-4-3-1-6-5
1-6-5-2-3-4


The first action is None because the 'X' is manually placed as the initial state in the algorithm definition. The final state shows that the example problem was solved quickly

## Visualization of the graph

In [ ]:
from simpleai.search.viewers import WebViewer
my_viewer = WebViewer()
my_problem = sudoku_solver(initial_state=initial_state)
result = astar(my_problem, viewer=my_viewer)

Starting the WebViewer, access it from your web browser, navigating to the address:
http://localhost:8000
To stop the WebViewer, use the "Stop running" link (on the viewer site, from the browser)
 * Serving Flask app 'simpleai.search.web_viewer_server'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://192.168.29.163:8000
Press CTRL+C to quit
127.0.0.1 - - [21/Oct/2023 00:50:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2023 00:50:23] "GET /static/styles.css HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2023 00:50:23] "GET /static/jquery-2.0.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2023 00:50:23] "GET /static/angular.min.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2023 00:50:23] "GET /static/EventSource.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2023 00:50:23] "GET /static/jquery.panzoom.min.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2023 00:50:23] "GET /static/main.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2023 00:50:24] "GET /graph HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2023 00:50:24] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [21/Oct/2023 00:50:24] "GET /event_stream HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2023 00:50:24] "GET /graph?unique=1697829624283 HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2

If main.py is executed using these lines, SimpleAI will mount a server on localhost to display the algorithm execution state tree step by step.

When accessing the localhost path, a view like the following will be displayed: